# unusable for now

In [ ]:
import numpy  as np
import pandas as pd
from   scipy.interpolate import interp1d
from   scipy.interpolate import interp2d
from   scipy.misc import derivative
from   scipy import optimize
import astropy.units as u
# import astropy.constants as c
from   astropy.cosmology import FlatLambdaCDM, z_at_value
from   tqdm import *
from   sympy import *
from   astropy.cosmology import Planck13 as cosmo
from   astropy import constants as const
import sys
from   scipy.interpolate import interp1d
from   scipy.interpolate import interp2d
from   scipy.special import zeta
import pickle

import matplotlib.pyplot as plt
from   matplotlib import ticker
from matplotlib import gridspec
import matplotlib.pylab as pylab

sys.path.insert(1, '../packages')
import units

import time



from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')



In [ ]:
z_ary_test  = np.logspace(-6,8,10000)
Xe_ary_test = Xe_interp(z_ary_test)

# Finite difference calculation of dXe/dz
dXe_dz_ary_test = derivative( Xe_interp,  z_ary_test[:-1],  dx= 1e-2 )


plt.figure(figsize=(8,5))


plt.plot( z_ary_test[:-1],  np.abs( dXe_dz_interp(z_ary_test[:-1]) ), label = r'$dX_e/dz$ (finite diff)', color = 'blue' )
plt.plot( z_ary_test[:-1],  np.abs( dXe_dz_ary_test ), label = r'$dX_e/dz$ (use "derivative")', color = 'magenta',linestyle='dotted' )


plt.xlabel(r'$z$')
plt.ylabel(r'$dX_e/dz$')

plt.xscale('log')
plt.yscale('log')

# plt.ylim(1-1e-3, 1+1e-3)

plt.legend()

In [ ]:
z_ary_test  = np.logspace(-2,8,10000)
Xe_ary_test = Xe_interp(z_ary_test)

# Finite difference calculation of dXe/dz
dXe_dz_ary_test = derivative( Xe_interp,  z_ary_test[:-1],  dx= 1e-2 )

# Regulator of dXe/dz difference
Reg_dXedz_diff = 10**(-10)

print('Regulator of dXe/dz difference = ', Reg_dXedz_diff)

plt.figure(figsize=(8,5))

plt.plot( z_ary_test[:-1], np.abs( dXe_dz_interp(z_ary_test[:-1])-dXe_dz_ary_test )/( np.abs( dXe_dz_interp(z_ary_test[:-1]) ) + Reg_dXedz_diff ), color = 'green' )
plt.plot( z_ary_test, 10**(-2) * np.ones(len(z_ary_test)), color = 'black', label = r'error = $10^{-2}$', linestyle='dashed' )
plt.plot( z_ary_test, 10**(-3) * np.ones(len(z_ary_test)), color = 'gray', label = r'error = $10^{-3}$', linestyle='dashed' )


plt.title(r'$|$finite diff$-$interp$|$/$(|$interp$|$$+$Reg$)$, Reg$=10^{-10}$')

plt.xlabel(r'$z$')
plt.ylabel(r'Relative Error of $dX_e/dz$')

plt.xscale('log')
plt.yscale('log')


plt.ylim(1e-17, 10)



plt.legend()

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# Test of the Code: z_cross
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

# mAp_test        = 10**(-11)  # in eV
# omega0_test     = 10*TCMB_0  # in eV

mAp_test        = 3.814*10**(-13)  # in eV
omega0_test     = 5*TCMB_0  # in eV


z_ary_plt   = np.logspace(-3, np.log10(2*10**6), 10**3)

x_A_plt = 0
x_B_plt = 3
x_C_plt = 5

plt.figure(figsize=(8,5))

plt.plot( z_ary_plt, np.where( mAsq(z_ary_plt, x_A_plt*TCMB_0)>=0,mAsq(z_ary_plt, x_A_plt*TCMB_0),0  )**(1/2) , label = r'$x=\omega/T=0$', color = 'red' )
plt.plot( z_ary_plt, np.where( mAsq(z_ary_plt, x_B_plt*TCMB_0)>=0,mAsq(z_ary_plt, x_B_plt*TCMB_0),0  )**(1/2) , label = r'$x=\omega/T=3$', color = 'blue' )
plt.plot( z_ary_plt, np.where( mAsq(z_ary_plt, x_C_plt*TCMB_0)>=0,mAsq(z_ary_plt, x_C_plt*TCMB_0),0  )**(1/2) , label = r'$x=\omega/T=5$', color = 'green' )

plt.plot( z_ary_plt, mAp_test * np.ones( len(z_ary_plt) ), label = r'$m_{A^\prime}$', color = 'black',linestyle='dashed' )


plt.title(r'Plasmon Mass Evolution')

plt.xlabel(r'$z$')
plt.ylabel(r'$m_{\gamma}$ [eV]')

plt.xscale('log')
plt.yscale('log')

plt.legend()


z_grid_ary_test = np.logspace(-3, 7, 1000)  # 

t1=time.time()
z_cross_ary_test = get_z_crossings(mAp_test, omega0_test,z_grid_ary_test)
t2=time.time()

print('m_Ap                 = ', mAp_test, 'eV')
print('omega0/TCMB_0        = ', omega0_test/TCMB_0)
print('')
print('t2-t1                = ', t2-t1, 's')
print('')
print('z_cross              = ', z_cross_ary_test)
print('')
print('Double check the results: ')
print('mA(z_cross)^2/mAp^2  = ', mAsq( z_cross_ary_test , omega0_test)/mAp_test**2)
print('')
print('<<<-----------------Attention----------------->>>')
print('The roubustness of the z_cross solver breaks down when $m_{A^\prime}<10^{-18}$ eV!!!')
print('This is fine because this is out of the relevant mAp range of FIRAS/PIXIE')

In [ ]:
z_ary_test = np.logspace(-3,3,100000)

x_test = 3

print('x = omega_0/TCMB_0 = ', x_test)

mAsq_ary_test = mAsq(z_ary_test, x_test*TCMB_0)

dmAsq_over_dz_ary_test = np.diff(mAsq_ary_test)/np.diff(z_ary_test)

plt.figure(figsize=(8,5))

plt.plot( z_ary_test[:-1], dmAsq_over_dz(z_ary_test[:-1], x_test*TCMB_0) , label = r'Analytical+$X_e$ interpolation', color = 'red' )
plt.plot( z_ary_test[:-1], dmAsq_over_dz_ary_test , label = r'Finite difference of $m_\gamma^2$', color = 'blue', linestyle='dashed' )

font = {'family': 'serif',
        'color':  'black',
        'weight': 'normal',
        'size': 12,}

plt.xlabel(r'$z$')
plt.ylabel(r'$dm^2_{\gamma}/dz$ [eV$^2$]', fontdict=font)

plt.xscale('log')
plt.yscale('log')

plt.legend()

In [ ]:

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# Test of the Code: P(A->Ap), I
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

mAp_test    = 10**(-5) # in eV

omega0_test = 1*TCMB_0
x_test      = omega0_test/TCMB_0

print( 'mAp            = ', mAp_test, 'eV')
print( 'omega0/TCMB_0  = ', omega0_test/TCMB_0)
print( '')
print( 'P_exact/P_RAD  = ', P_over_eps2(mAp_test, x_test)/P_over_eps2_RAD(x_test) )

t1 = time.time()
print( 'P_exact/eps^2  = ', P_over_eps2(mAp_test, x_test) )
t2 = time.time()
print( '')
print('Time of P_exact = ', t2-t1, 's')


# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# Test of the Code: P(A->Ap), II
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

mAp_ary_test = np.linspace(1e-9,1e-4,1000)  # in eV

x_ary = 1

P_ratio_ary_test = [ P_over_eps2(mAp, x_ary)/P_over_eps2_RAD(x_ary) for mAp in mAp_ary_test]

plt.figure(figsize=(8,5))

plt.plot( mAp_ary_test, P_ratio_ary_test , label = r'$P_{\gamma \rightarrow A^\prime}$(Exact)$/$$P_{\gamma \rightarrow A^\prime}$(RAD)', color = 'blue' )
plt.plot( mAp_ary_test, np.ones(len(mAp_ary_test)) , label = r'Ratio$=1$', color = 'black', linestyle='dashed' )


font = {'family': 'serif',
        'color':  'black',
        'weight': 'normal',
        'size': 12,}

plt.xlabel(r'$z$'  , fontdict=font)
plt.ylabel(r'Ratio', fontdict=font)

plt.xscale('log')
# plt.yscale('log')

plt.ylim(1-0.2, 1+0.2)
# plt.ylim(1-0.5, 1+0.5)
# plt.ylim(0, 2)

plt.legend()

In [ ]:

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# Test of the Code (1): Check of P/eps^2 from interp (Point Check)
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

mAp_test = 1e-13  # eV
x_test   = 0.3


P_over_eps2_exact_test  = P_over_eps2( mAp_test, x_test )
P_over_eps2_interp_test = P_over_eps2_interp( mAp_test, x_test )[0]

print('----------------------Point Check----------------------')
print('mAp_test          = ', mAp_test, 'eV')
print('x_test            = ', x_test)
print('')
print( 'P/eps^2 (exact)  = ', P_over_eps2_exact_test )
print( 'P/eps^2 (interp) = ', P_over_eps2_interp_test )
print('')
print('relative error    = ', np.abs(P_over_eps2_exact_test-P_over_eps2_interp_test)/P_over_eps2_interp_test )

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# Test of the Code (2): Check of P/eps^2 from interp (Scan Check)
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

N_x_check   = 35
N_mAp_check = 30

x_check_ary   = np.linspace( x_P_min, x_P_max, N_x_check )
mAp_check_ary = np.logspace( np.log10(mAp_P_min), np.log10(mAp_P_max), N_mAp_check )


interp_error_2Dary = np.zeros( (len(x_check_ary),len(mAp_check_ary)) )

for j in range(0,len(mAp_check_ary)):
    
    for i in range(0,len(x_check_ary)):
        
        x_i   = x_check_ary[i]
        mAp_j = mAp_check_ary[j]   # in eV
        
        P_over_eps2_calc_ij = P_over_eps2(mAp_j, x_i)
        
        P_over_eps2_interp_ij = P_over_eps2_interp(mAp_j, x_i)[0]
        
#         print('P/eps^2         = ', P_over_eps2_calc_ij)
#         print('P/eps^2(interp) = ', P_over_eps2_interp_ij)
#         print('')
        
        if ( P_over_eps2_interp_ij + P_over_eps2_calc_ij )>= 0.002 :
            
            interp_error_2Dary[i][j] = 2*( np.abs(P_over_eps2_interp_ij-P_over_eps2_calc_ij) )/( P_over_eps2_interp_ij + P_over_eps2_calc_ij )
        
        else:
            
            interp_error_2Dary[i][j] = 0
            
# The maximum (relative) error
error_max =  np.max(interp_error_2Dary)

print('')
print('----------------------Scan Check----------------------')
print('mAp_min                = ', mAp_P_min, 'eV')
print('mAp_max                = ', mAp_P_max, 'eV')
print('')
print('x_min                  = ', x_P_min)
print('x_max                  = ', x_P_max)
print('')
print('N_mAp_check            = ', N_mAp_check)
print('N_x_check              = ', N_x_check)
print('')
print('maximum relative error = ', error_max)

In [ ]:

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# Test of the Code chi^2
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

mAp_test   = 1e-13  # eV
eps_test   = 10**(-5)
    
print( 'mAp    = ', mAp_test, 'eV')
print( 'eps    = ', eps_test)
print( '')
t1 = time.time()
print( 'chi^2 (Fix   T0=TCMB_0)    = ', chi_sq_P(  mAp_test , eps_test, TCMB_0 ) )
t2 = time.time()
print( 'chi^2 (Vary  T0)           = ', chi_sq_minT0_P( mAp_test, eps_test ) )
t3 = time.time()
print( 'chi^2 (Vary  T0, Vary eps) = ', chi_sq_minT0_mineps_P(mAp_test) )
t4 = time.time()
print( '')

print( 'Time of chi^2 (Fix   T0=TCMB_0)    = ', t2-t1, 's')
print( 'Time of chi^2 (Vary  T0)           = ', t3-t2, 's')
print( 'Time of chi^2 (Vary  T0, Vary eps) = ', t4-t3, 's')

In [ ]:
# Import COBE-FIRAS data (From astro-ph/9605054)
import pandas as pd

FIRAS_data_ary = np.transpose( np.array( pd.read_csv('../data/FIRASData.csv') ) )

# nu [cm^-1]
FIRAS_nu_ary  = np.ndarray.flatten( FIRAS_data_ary[[0]] )

# Intensity [MJy * sr^-1]
FIRAS_I0_ary  = np.ndarray.flatten( FIRAS_data_ary[[1]] )

# Residual [kJy * sr^-1]
FIRAS_res_ary   = np.ndarray.flatten( FIRAS_data_ary[[2]] )

# Uncertainty [kJy * sr^-1]
FIRAS_sigma_ary = np.ndarray.flatten( FIRAS_data_ary[[3]] )


# Q array  ('3.3 Variance Estimation')
Q_ary = np.array([1.000, 0.176, -0.203, 0.145, 0.077, -0.005, -0.022, 0.032,
                          0.053, 0.025, -0.003, 0.007, 0.029, 0.029, 0.003, -
                          0.002, 0.016, 0.020, 0.011, 0.002, 0.007,
                          0.011, 0.009, 0.003, -0.004, -0.001, 0.003, 0.003, -
                          0.001, -0.003, 0.000, 0.003, 0.009, 0.015,
                          0.008, 0.003, -0.002, 0.000, -0.006, -0.006, 0.000, 0.002, 0.008])

# Length of FIRAS data
len_FIRAS = len(FIRAS_nu_ary)

# Smallest frequency in FIRAS data [cm^-1]
FIRAS_nu_min = FIRAS_nu_ary[0]

# Largest frequency in FIRAS data [cm^-1]
FIRAS_nu_max= FIRAS_nu_ary[-1]

# ----------------------------------------------------------------------
# Calc: Cov[i,j] = Q(|i-j|) * sig_i * sig_j
# ----------------------------------------------------------------------

# Covariance Matrix: Initialization
Cov   = np.zeros((len_FIRAS, len_FIRAS))
Cov_2 = np.zeros((len_FIRAS, len_FIRAS))


# Def Covariance by X.G
for i in range(len_FIRAS):
    
    for j in range(len_FIRAS):
        
        # Index of Q array
        # More intuitively, this is the 'distance' between nu_i and nu_j
        idx_Q = np.abs(i-j)
        
        # Q value
        Q_value = Q_ary[idx_Q]
        
        # Cov_ij
        Cov[i,j] = FIRAS_sigma_ary[i] * FIRAS_sigma_ary[j] * Q_value
        

# Def Covariance by Sid
# See https://github.com/smsharma/dark-photons-perturbations/blob/master/grf/grf.py

# array of nu - nu^prime
nu_minus_nu_prime_ary = np.linspace( FIRAS_nu_min , FIRAS_nu_max, len_FIRAS ) - FIRAS_nu_min


for i in range(len_FIRAS):
    
    for j in range(len_FIRAS):
        
        # nu_i - nu_j
        nu_minus_nu_prime = np.abs( FIRAS_nu_ary[i] - FIRAS_nu_ary[j] )
        
        # Index of Q array
        # More intuitively, this is the 'distance' between nu_i and nu_j
        # .argmin() can return the index of the minimal value
        idx_Q = np.abs( nu_minus_nu_prime_ary - nu_minus_nu_prime ).argmin()
        
        # Q value
        Q_value = Q_ary[idx_Q]
        
        # Cov_ij
        Cov_2[i,j] = FIRAS_sigma_ary[i] * FIRAS_sigma_ary[j] * Q_value
        
